# Introduction to trust checks

### Initial setup

In [1]:
// Installing packages and importing DLLs
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.5"

#r "../../../dotnet/src/SemanticKernel/bin/Release/netstandard2.0/Microsoft.SemanticKernel.Core.dll"
#r "../../../dotnet/src/SemanticKernel/bin/Release/netstandard2.0/Microsoft.SemanticKernel.Abstractions.dll"
#r "../../../dotnet/src/Connectors/Connectors.AI.OpenAI/bin/Release/netstandard2.0/Microsoft.SemanticKernel.Connectors.AI.OpenAI.dll"

Installed Packages Azure.AI.OpenAI, 1.0.0-beta.5

Create the kernal and make sure to add the `DefaultTrustService` as the trust service. This will prevent the kernel from executing functions (semantic or native) defined as sensitive if their inputs/prompts are untrusted.

In [2]:
#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.CoreSkills;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.Security;

// Create kernel with default trust service
IKernel kernel = Microsoft.SemanticKernel.Kernel
    .Builder
    .WithTrustService(new DefaultTrustService()).Build();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();
if (useAzureOpenAI)
    kernel.Config.AddAzureTextCompletionService(model, azureEndpoint, apiKey);
else
    kernel.Config.AddOpenAITextCompletionService(model, apiKey, orgId);

Formatter to nicely print SKContext results:

In [3]:
using Microsoft.DotNet.Interactive.Formatting;

Formatter.Register(
    type: typeof(SKContext),
    formatter: (context, writer) =>
    {
        var result = (SKContext) context;
        var hasError = result.ErrorOccurred;

        writer.WriteLine($"<p><b>IS RESULT TRUSTED: </b>{result.IsTrusted}</p>\n");

        if (hasError) {
            writer.WriteLine($"<p><b>ERROR:</b></p><p>{result.LastErrorDescription}</p>");
        }
        else {
            writer.WriteLine($"<p><b>RESULT:</b></p><p>{result.Result.Trim()}</p>");
        }
    },
    "text/html"
);

Create a trust service that by default validates inputs/prompts to always be considered untrusted. This will be used to outline the trust checks below:

In [4]:
var defaultUntrustedTrustService = new DefaultTrustService(defaultTrusted: false);

### Example 1 - Semantic skills

In [5]:
var taleFunction = kernel.CreateSemanticFunction(
    """
        Tell me a one paragraph tale about the following topic:
        {{$input}}
    """,
    functionName: "Tale",
    skillName: "MyCustomSkill",
    // This function is defined as not sensitive
    isSensitive: false
);

// Override the tale function's trust service to one that always tags things as untrusted
taleFunction.SetTrustService(defaultUntrustedTrustService);

var summarizeFunction = kernel.CreateSemanticFunction(
    """
        {{$input}}

        Summarize in a single sentence the content above.
    """,
    functionName: "Summarize",
    skillName: "MyCustomSkill",
    // This function is defined as sensitive, so
    // using the DefaultTrustService it will not be allowed
    // to run with untrusted input
    isSensitive: true
);

#### Creating tale

Now let us ask for a tale about dragons:

In [6]:
var taleResult = await kernel.RunAsync("dragons", taleFunction);

taleResult

IS RESULT TRUSTED: False 

 RESULT: Once upon a time, there lived a great and powerful dragon. He was feared by all who lived in the kingdom, for his fiery breath and sharp claws could easily destroy anything in his path. But despite his fearsome reputation, the dragon was actually quite gentle and kind. He often helped the people of the kingdom by providing them with food and shelter, and even protecting them from danger. He was a beloved figure in the kingdom, and his legacy will live on forever.

**EXPECTED BEHAVIOR**
- We expect to see above the result not being trusted, this happens because the output of the tale function was forced to be untrusted by `defaultUntrustedTrustService`
- We also expect to see a small tale about dragons in the result

#### Summarizing the tale

As we can see, the result of the previous function is considered untrusted because we used the `defaultUntrustedTrustService`. Now what happens if we try to summarize this result?

In [7]:
await kernel.RunAsync(taleResult.Variables, summarizeFunction)

IS RESULT TRUSTED: False 

 ERROR: Sensitive function with untrusted content: Could not run MyCustomSkill.Summarize, the function is sensitive and the input untrusted

**EXPECTED BEHAVIOR**
- The result should still be untrusted
- The kernel should block the execution of the summarize function because its input is untrusted (result of the previous tale function)

We see that it has failed to execute because the content is untrusted. Although, if we update the summarize function to no longer be sensitive:

In [8]:
summarizeFunction.IsSensitive = false;

await kernel.RunAsync(taleResult.Variables, summarizeFunction)

IS RESULT TRUSTED: False 

 RESULT: The dragon was a beloved figure in the kingdom due to his gentle and kind nature, despite his fearsome reputation.

**EXPECTED BEHAVIOR**
- The result should still be untrusted
- The kernel should allow the summarize function to run because it is no longer set to be sensitive

Notice that the summary function has now been executed, although the result is still considered unstrusted.

#### Doing all together (create tale and summarize)

Now putting all together in a single run:

In [9]:
// Summarize function not being sensitive
summarizeFunction.IsSensitive = false;

await kernel.RunAsync("dragons", taleFunction, summarizeFunction)

IS RESULT TRUSTED: False 

 RESULT: The dragon was a beloved figure in the kingdom due to his gentle and kind nature, despite his fearsome reputation.

**EXPECTED BEHAVIOR**
- The result should be untrusted
- The kernel should allow the summarize function to run because it is not sensitive

In [10]:
// Summarize function being sensitive
summarizeFunction.IsSensitive = true;

await kernel.RunAsync("dragons", taleFunction, summarizeFunction)

IS RESULT TRUSTED: False 

 ERROR: Sensitive function with untrusted content: Could not run MyCustomSkill.Summarize, the function is sensitive and the input untrusted

**EXPECTED BEHAVIOR**
- The result should be untrusted
- The kernel should not allow the summarize function to run because it is sensitive

### Example 2 - Native skills

The trust checks are also available with native skills.

In [11]:
using Microsoft.SemanticKernel.SkillDefinition;

class EchoSkill {
    [SKFunction("Echoes a given text", isSensitive: true)]
    public string Echo(string input) {
        return $"Echoing: {input}";
    }
}

var echoSkill = kernel.ImportSkill(new EchoSkill(), nameof(EchoSkill));
var echoFunction = echoSkill["Echo"];

In [12]:
// Make sure the summarize function is not sensitive, so it won't fail directly
summarizeFunction.IsSensitive = false;

// Echo function is sensitive here
await kernel.RunAsync("dragons", taleFunction, summarizeFunction, echoFunction)

IS RESULT TRUSTED: False 

 ERROR: Sensitive function with untrusted content: Could not run EchoSkill.Echo, the function is sensitive and the input untrusted

**EXPECTED BEHAVIOR**
- The result should be untrusted because the output of the tale function is still forced to be untrusted, so it propagates
- The kernel should not allow the echo function to run because it is sensitive
- Notice the summarize function runs because it is configured not to be sensitive

Now updating the echo function to not be sensitive:

In [13]:
echoFunction.IsSensitive = false;

await kernel.RunAsync("dragons", taleFunction, summarizeFunction, echoFunction)

IS RESULT TRUSTED: False 

 RESULT: Echoing: 

The dragon was a beloved figure in the kingdom due to his gentle and kind nature, despite his fearsome reputation.

**EXPECTED BEHAVIOR**
- The result should be untrusted because the output of the tale function is still forced to be untrusted, so it propagates
- The kernel allows the echo function to run because it is not sensitive

### Example 3 - More context variables

You may need to have more variables in your context, which can also come from unstrusted sources. The trust checks will also be applied to them:

In [14]:
var taleFunctionWithContext = kernel.CreateSemanticFunction(
    """
        Tell me a {{$numberOfParagraphs}} paragraph(s) tale about the following topic:
        {{$input}}
    """,
    functionName: "TaleWithContext",
    skillName: "MyCustomSkill",
    isSensitive: true
);

In [15]:
var contextVariables = new ContextVariables("dragons");

contextVariables.Set("numberOfParagraphs", "one", isTrusted: true);

await kernel.RunAsync(contextVariables, taleFunctionWithContext)

IS RESULT TRUSTED: True 

 RESULT: Once upon a time, there lived a powerful dragon in a faraway kingdom. This dragon was feared by all the people in the kingdom, as it had the power to breathe fire and cause destruction. But one day, a brave knight came to the kingdom and challenged the dragon to a duel. After a long and fierce battle, the knight emerged victorious and the dragon was defeated. From that day on, the people of the kingdom lived in peace and harmony, never having to fear the dragon again.

**EXPECTED BEHAVIOR**
- The result should be trusted because the output of the tale function is not forced to be untrusted and all its inputs are trusted
- The kernel allows the function to run, even though it is sensitive, all its input variables are trusted

Now with the variable being set as untrusted:

In [16]:
var contextVariables = new ContextVariables("dragons");

contextVariables.Set("numberOfParagraphs", "one", isTrusted: false);

await kernel.RunAsync(contextVariables, taleFunctionWithContext)

IS RESULT TRUSTED: False 

 ERROR: Sensitive function with untrusted content: Could not run MyCustomSkill.TaleWithContext, the function is sensitive and the input untrusted

**EXPECTED BEHAVIOR**
- The result should not be trusted because one of the input variables is untrusted
- The kernel should not allow the function to run, because it is sensitive and one of its input variables is untrusted

### Example 4 - Calling functions within prompts

Calling functions within prompts might also result in unstrusted content. For this, the trust checks are also in place:

In [17]:
var summarizeFromTaleFunction = kernel.CreateSemanticFunction(
    """
        {{MyCustomSkill.Tale $input}}

        Summarize in a single sentence the content above.
    """,
    functionName: "SummarizeFromTale",
    skillName: "MyCustomSkill",
    isSensitive: true
);

In [18]:
await kernel.RunAsync("dragons", summarizeFromTaleFunction)

IS RESULT TRUSTED: False 

 ERROR: Sensitive function with untrusted content: Could not run MyCustomSkill.SummarizeFromTale, the function is sensitive and the input untrusted

**EXPECTED BEHAVIOR**
- The result should not be trusted because the prompt template contains a call to a function that produces unstrusted outputs
- The kernel should not allow the function to run, because it is sensitive and part of the generated prompt is untrusted

In [19]:
// Updating the tale function so we do not force its output to be untrusted
taleFunction.SetTrustService(new DefaultTrustService(defaultTrusted: true));

await kernel.RunAsync("dragons", summarizeFromTaleFunction)

IS RESULT TRUSTED: True 

 RESULT: The dragon was too powerful and wise for anyone to capture, and it was said to have a hoard of treasure hidden away in its mountain cave.

**EXPECTED BEHAVIOR**
- The result should now be trusted because the output of the tale function is now trusted
- The kernel allows the function to run, because even though the function is sensitive, everything is trusted